### Меня зовут Сергей Казьмин
### Моя версия Python: 3.6.6

##  ---------------------------------------------------------UPDATE---------------------------------------------------------
###### Мне сообщили о следующих замечаниях:
1. Не учтены запросы латиницей русскими (изначальна подразумевалась разбивка по языку девайса, указанному в столбце locale)

2. Невозможно масштабирование на большее количество языков, нужно учесть такую возможность. 

3. В результат необходимо добавить количество запросов (достопримечательность - 5000, где поесть - 4000 и т.д.)

4. В реальности нет возможности загрузить весь датасет в память и обработка ведётся последовательно событие за событием. Как изменится реализация? 

5. Опционально: Исключить из общего списка запросы по категориям и вывести их отдельно. Найти их можно в приложении в поиске на вкладке «категории». Чтобы посмотреть как они выглядят на английском, необходимо сменить язык телефона на английский и перезапустить приложение. 



### 4 пункт
Я недопонял, надо ли мне в этом пункте что-то исправлять в коде или просто нужно ответить на вопрос словами :)

Я напишу свои мысли, как это реализовать. Если вдруг надо было писать код - пожалуйста, сообщите мне.

Итак
- Если памяти не хватает под весь датасет, то можно, во-первых, воспользоваться параметром chunksize функции pandas.read_csv, принцип работы с датасетом не изменится, мы просто будем итерироваться по chunk-ам и проделывать аналогичные действия. 
- Другой вариант заключается в следующем. В моих циклах for мы обычно сравниваем только i и i+1 события, после чего решаем оставить событие i в датасете или исключить его. Причём сохраняем именно индекс, который занимает мало памяти. То есть на каждой итерации нам достаточно хранить в памяти i и i+1 события. Можно читать из файла построчно, открывать это всё в pandas, т.е. будет dataframe из 2 строк, проводить их сравнение и оставлять индекс. Потом по индексам выгружать необходимые события, делать из них словарь частоты встречаемости. ГЛАВНОЕ, надо просто получить первичный словарь встречаемости, например, такой же, какой я получаю в конце этого скрипта. После того, как сделан этот первый словарь, действия такие:
    1. i запрос получен, получаем i+1
    2. если i+1 стал короче, чем i, то берём i запрос (это алгоритм, до которого додумался я :) ) 
    3. теперь ищем в нашем словаре встречаемости ключ, максимально похожий по метрике Джаро-Винклера на i-ый запрос и увеличиваем значение по этому ключу на единицу. (Можно попробовать ускорить это, начиная искать не среди всех ключей, а среди ключей с большой частотой встречаемости).
    4. Если не нашли достаточно похожтй ключ (надо установить порог, начиная с которого можно считать похожими), то добавляем новый ключ со значением 1.
    5. В итоге наш словарь встречаемости будет отражать текущую ситуацию популярных запросов.

#### Я внёс исправления, прокомментирую по пунктам

1. Сделал разбивку по столбцу locale - теперь словарь встречаемости выглядит так (query, language):frequency
2. Теперь можно отмасштабировать на большее количество языков, просто будем добавлять больше меток в словарь встречаемоcти
3. Добавил в результат количество запросов
4. Выше 
5. Добавил после первых двух исходных ТОПов

##  ----------------------------------------------------UPDATE ENDED----------------------------------------------------

##### Сначала опишу задание так, как я его понял.
В датасете собраны логи действий юзера при набирании запроса, например:
0. п
1. пр
2. про
3. прод
4. ...
7. продукты

Здесь запрос пользователя - это слово "продукты", в то время как "п", "пр", "про", "прод", ... - это побочная информация. 
#### Моя цель в следующем: выделить в логах из действий пользователя конкретный запрос и сформировать ТОП.
В случае моего примера из всех данных о вводе запроса надо выделить строку, в которой написано "продукты".
Чтобы выделять отдельные запросы следует подготовить данные. Стоит отметить, что определённая погрешность присутствует в любом моём действии с данными, но, если учесть, что датасет очень велик, в конце должен получиться достаточно достоверный результат.

### Закономерности, которые я буду использовать при обработке датасета
1. Если идти по колонке query сверху вниз, длина слов, набранных пользователем, сначала возрастает, потом достигает максимальной длины, когда это ЗАПРОС пользователя и потом либо убывает, либо вообще начинаются данные о запросе другого юзера.
2. Продолжительность набора запроса пользователем примерно 2-8 секунд, следовательно, можно группировать данные по id юзера и по интервалу времени 7-10 секунд. Получив интервал, можно выбрать из него слово, максимальное по длине, и с большой вероятностью оно окажется искомым запросом.

### Время, за которое скрипт обрабатывает данные ~ 50 минут на моём ноутбуке

In [1]:
import pandas as pd
import datetime
import numpy as np
import jellyfish
import operator

In [2]:
data = pd.read_csv("raw_search_data.csv", sep=';', error_bad_lines=False) 
#Ошибки возникают с длинными запросами, в которых встречаются кавычки и точки с запятыми, очевидно, 
#таких запросов немного, проигнорировать можно

b'Skipping line 22414: expected 5 fields, saw 6\nSkipping line 33796: expected 5 fields, saw 6\nSkipping line 33797: expected 5 fields, saw 6\nSkipping line 33798: expected 5 fields, saw 6\nSkipping line 36710: expected 5 fields, saw 6\n'
b'Skipping line 922233: expected 5 fields, saw 6\n'
b'Skipping line 3615615: expected 5 fields, saw 6\n'
b'Skipping line 4153368: expected 5 fields, saw 6\nSkipping line 4153369: expected 5 fields, saw 6\nSkipping line 4153370: expected 5 fields, saw 6\n'


In [3]:
#Убираем все строки, в которых есть незаполненные столбцы. Их немного, не сильно повлияет на результат
data = data.dropna() 

data['query'] = data['query'].apply(lambda x: str(x).lower()) 

#Игнорируем запросы длиннее 50 символов, таких всего ~10000
data = data[list(map(lambda x: len(x) < 50, data[:]['query']))]

In [4]:
#Преобразуем дату-время из строки в datetime
data['datetime'] = data['datetime'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [5]:
data = data.reset_index()
data = data.drop(columns=['index'])

In [6]:
idxs_for_save = []
#Алгоритм, отбрасывающий больше 50% данных о том, как пользователи вводили запрос. 
#Пока длина запроса увеличивается ничего не делаем. 
#Как только в следующей строке лога длина запроса уменьшилась - сохраняем индекс "локального максимума" длины запроса.
#Алгоритм позволяет отбросить большую часть промежуточных запросов и оставить только конечные.
for i in range(data.shape[0] - 1):
    if len(str(data.iloc[i+1]['query'])) <= len(str(data.iloc[i]['query'])):
        idxs_for_save.append(i)

In [7]:
filtered_data = data.iloc[idxs_for_save]

#Удаляем запросы длиной не больше 3 символов
filtered_data = filtered_data[filtered_data[:]['query'].apply(lambda x: len(x)) > 2]

filtered_data = filtered_data.reset_index()
filtered_data = filtered_data.drop(columns=['index'])

In [8]:
#Функция сравнивает две строки метрикой Джаро-Винклера
#Выдаёт True, если значение метрики больше или равно порога limit, False иначе
def check_string_similarity(str_1, str_2, limit):
    return jellyfish.jaro_winkler(str_1, str_2) >= limit

In [9]:
idxs_for_delete = []
#Алгоритм сравнивает две соседние строчки в логе
#Если это запросы от одного юзера, и они были отправлены в интервале 5 секунд, и при этом они сильно похожи по нашей метрике
#То это один и тот же запрос => можно удалить данные об одном из них
for i in range(filtered_data.shape[0] - 1):
    if filtered_data.iloc[i]['uid'] == filtered_data.iloc[i+1]['uid'] and \
            (max(filtered_data.iloc[i]['datetime'], filtered_data.iloc[i+1]['datetime']) - \
            min(filtered_data.iloc[i]['datetime'], filtered_data.iloc[i+1]['datetime'])) < datetime.timedelta(seconds=5) and \
            check_string_similarity(filtered_data.iloc[i]['query'], filtered_data.iloc[i+1]['query'], 0.93):
        
        if len(filtered_data.iloc[i]['query']) <= len(filtered_data.iloc[i+1]['query']):
            idxs_for_delete.append(i)
        else:
            idxs_for_delete.append(i+1)

In [10]:
idx_for_delete = sorted(list(set(idxs_for_delete)))
filtered_data = filtered_data.drop(index=idxs_for_delete)
filtered_data = filtered_data.reset_index()
filtered_data = filtered_data.drop(columns=['index'])

In [11]:
filtered_data.to_csv("filtered_data.csv", sep=';', index=False)

In [12]:
rus_lang = []
eng_lang = []

# Сортируем все метки из locale по rus и eng
for lang in pd.unique(filtered_data['locale']):
    if lang[:2] == 'en' or lang[:2] == 'En':
        eng_lang.append(lang)
    else:
        rus_lang.append(lang)

In [13]:
# Заменяем метки из locale на две ru и en 
filtered_data['locale'] = filtered_data['locale'].apply(lambda x: 'en' if x in eng_lang else 'ru')

In [14]:
# список всех запросов вместе с метками
queries_list = list(zip(filtered_data['query'], filtered_data['locale']))

In [15]:
frequency_dict = {}
#Формируем dict встречаемости запросов вместе с метками
for query in queries_list:
    if query not in frequency_dict.keys():
        frequency_dict[query] = 1
    else:
        frequency_dict[query] += 1   

In [16]:
keys_more_50freq = []
keys_more_10freq = []

#Сформируем список ключей, встречающихся более 50 и 10 раз

for key in frequency_dict.keys():
    if frequency_dict[key] > 50:
        keys_more_50freq.append(key)
        
for key in frequency_dict.keys():
    if frequency_dict[key] > 10:
        keys_more_10freq.append(key)

In [17]:
keys_for_delete = [] 

# Найдём среди них похожие ключи, сформируем список для удаления ключей
for key in keys_more_50freq:
    for key_ in keys_more_10freq:
        if key != key_ and key[1] == key_[1] and check_string_similarity(key[0], key_[0], 0.92):
            if frequency_dict[key] >= frequency_dict[key_]:
                frequency_dict[key] += frequency_dict[key_] #встречаемось похожего уходит другому
                keys_for_delete.append(key_)                #удаляется менее встречающийся ключ
            else:
                frequency_dict[key_] += frequency_dict[key]
                keys_for_delete.append(key)

In [18]:
for key in set(keys_for_delete):
    frequency_dict.pop(key)

In [19]:
#сортируем по встречаемости
sorted_frequency = sorted(frequency_dict.items(), key=operator.itemgetter(1))

In [20]:
ru_frequency = []
eng_frequency = []

#разбиваем по меткам на rus и eng

for i in range(len(sorted_frequency))[::-1]:
    if sorted_frequency[i][0][1] == 'en':
        eng_frequency.append((sorted_frequency[i][0][0], sorted_frequency[i][1]))
    else:
        ru_frequency.append((sorted_frequency[i][0][0], sorted_frequency[i][1]))

In [21]:
top_100_rus_queries = ru_frequency[:100]
top_100_eng_queries = eng_frequency[:100]

## ТОП-100 запросов на русском

In [22]:
for query, i in zip(top_100_rus_queries, range(1, 101)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  достопримечательность .... 11207
2.  где поесть ............... 10187
3.  продукты ................. 8222
4.  гостиница ................ 6272
5.  заправка ................. 4261
6.  шоппинг .................. 3109
7.  аптека ................... 1658
8.  транспорт ................ 1640
9.  москва ................... 1594
10.  банкомат ................. 1489
11.  аэропорт ................. 1415
12.  метро .................... 1413
13.  минск .................... 1254
14.  супермаркет .............. 1204
15.  автовокзал ............... 1182
16.  рынок .................... 1142
17.  магазин .................. 1124
18.  туалет ................... 1053
19.  парковка ................. 1018
20.  макдональдс .............. 932
21.  улица .................... 862
22.  банк ..................... 816
23.  ночная жизнь ............. 795
24.  отдых с детьми ........... 752
25.  музей .................... 747
26.  вокзал ................... 721
27.  пляж ..................... 705
28.  самара ....

## ТОП-100 запросов на английском

In [23]:
for query, i in zip(top_100_eng_queries, range(1, 101)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  hotel .................... 6943
2.  where to eat ............. 6474
3.  sights ................... 3342
4.  groceries ................ 3189
5.  supermarket .............. 1747
6.  mcdonalds ................ 1591
7.  shopping ................. 1557
8.  airport .................. 1379
9.  atm ...................... 1374
10.  transport ................ 1126
11.  carrefour ................ 1082
12.  via ...................... 1054
13.  san ...................... 1038
14.  lidl ..................... 971
15.  santa .................... 958
16.  porto .................... 930
17.  pristina ................. 908
18.  starbucks ................ 889
19.  gas ...................... 862
20.  arkhangai ................ 852
21.  nightlife ................ 841
22.  ulaanbaatar .............. 767
23.  the ...................... 752
24.  bank ..................... 744
25.  mercado .................. 733
26.  camping .................. 709
27.  достопримечательность .... 695
28.  продукты ..........

In [24]:
rus_categories = ['где поесть', 'гостиница', 'продукты', 'достопримечательность', 'wifi', 'транспорт', 'заправка', \
                  'парковка', 'шоппинг', 'банкомат', 'ночная жизнь', 'отдых с детьми', 'банк', 'развлечения', \
                  'больница', 'аптека', 'полиция', 'туалет', 'почта']
eng_categories = ['where to eat', 'hotel', 'groceries', 'sights', 'wifi', 'transport', 'gas', 'parking', 'shopping', \
                  'atm', 'nightlife', 'family holiday', 'bank', 'entertainment', 'hospital', 'pharmacy', 'police', \
                  'toilet', 'post']

In [25]:
cat_dict = {}
# Словарь встречаемости категорий, при этом из общего словаря удаляем все категории
for cat in zip(rus_categories, eng_categories):
    cat_dict[(cat[0], 'ru')] = frequency_dict.pop((cat[0], 'ru'))
    cat_dict[(cat[1], 'en')] = frequency_dict.pop((cat[1], 'en'))

In [26]:
# Дальше вся та же обработка, что была с исходным словарём

sorted_cat_frequency = sorted(cat_dict.items(), key=operator.itemgetter(1))

cat_ru_frequency = []
cat_eng_frequency = []

for i in range(len(sorted_cat_frequency))[::-1]:
    if sorted_cat_frequency[i][0][1] == 'en':
        cat_eng_frequency.append((sorted_cat_frequency[i][0][0], sorted_cat_frequency[i][1]))
    else:
        cat_ru_frequency.append((sorted_cat_frequency[i][0][0], sorted_cat_frequency[i][1]))

In [27]:
sorted_without_cats_frequency = sorted(frequency_dict.items(), key=operator.itemgetter(1))

without_cats_ru_frequency = []
without_cats_eng_frequency = []

for i in range(len(sorted_without_cats_frequency))[::-1]:
    if sorted_without_cats_frequency[i][0][1] == 'en':
        without_cats_eng_frequency.append((sorted_without_cats_frequency[i][0][0], sorted_without_cats_frequency[i][1]))
    else:
        without_cats_ru_frequency.append((sorted_without_cats_frequency[i][0][0], sorted_without_cats_frequency[i][1]))

In [28]:
top_100_rus_queries_without_cats = without_cats_ru_frequency[:100]
top_100_eng_queries_without_cats = without_cats_eng_frequency[:100]

## ТОП категорий на русском

In [29]:
for query, i in zip(cat_ru_frequency, range(1, 20)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  достопримечательность .... 11207
2.  где поесть ............... 10187
3.  продукты ................. 8222
4.  гостиница ................ 6272
5.  заправка ................. 4261
6.  шоппинг .................. 3109
7.  аптека ................... 1658
8.  транспорт ................ 1640
9.  банкомат ................. 1489
10.  туалет ................... 1053
11.  парковка ................. 1018
12.  банк ..................... 816
13.  ночная жизнь ............. 795
14.  отдых с детьми ........... 752
15.  развлечения .............. 595
16.  почта .................... 491
17.  wifi ..................... 486
18.  больница ................. 445
19.  полиция .................. 150


## ТОП категорий на английском

In [30]:
for query, i in zip(cat_eng_frequency, range(1, 20)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  hotel .................... 6943
2.  where to eat ............. 6474
3.  sights ................... 3342
4.  groceries ................ 3189
5.  shopping ................. 1557
6.  atm ...................... 1374
7.  transport ................ 1126
8.  gas ...................... 862
9.  nightlife ................ 841
10.  bank ..................... 744
11.  wifi ..................... 642
12.  pharmacy ................. 627
13.  hospital ................. 614
14.  parking .................. 567
15.  toilet ................... 503
16.  entertainment ............ 298
17.  post ..................... 283
18.  family holiday ........... 263
19.  police ................... 195


## ТОП-100 запросов на русском без категорий

In [31]:
for query, i in zip(top_100_rus_queries_without_cats, range(1, 101)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  москва ................... 1594
2.  аэропорт ................. 1415
3.  метро .................... 1413
4.  минск .................... 1254
5.  супермаркет .............. 1204
6.  автовокзал ............... 1182
7.  рынок .................... 1142
8.  магазин .................. 1124
9.  макдональдс .............. 932
10.  улица .................... 862
11.  музей .................... 747
12.  вокзал ................... 721
13.  пляж ..................... 705
14.  самара ................... 665
15.  площадь .................. 623
16.  торговый центр ........... 590
17.  прага .................... 568
18.  парк ..................... 561
19.  крым ..................... 548
20.  санкт петербург .......... 539
21.  кафе ..................... 538
22.  озеро .................... 533
23.  lidl ..................... 505
24.  киев ..................... 503
25.  краснодар ................ 494
26.  отель .................... 476
27.  водопад .................. 462
28.  халкидики ..............

## ТОП-100 запросов на английском без категорий

In [32]:
for query, i in zip(top_100_eng_queries_without_cats, range(1, 101)):
    print(str(i) + '. ', query[0], str('.' * (25 - len(query[0]))), str(query[1]))

1.  supermarket .............. 1747
2.  mcdonalds ................ 1591
3.  airport .................. 1379
4.  carrefour ................ 1082
5.  via ...................... 1054
6.  san ...................... 1038
7.  lidl ..................... 971
8.  santa .................... 958
9.  porto .................... 930
10.  pristina ................. 908
11.  starbucks ................ 889
12.  arkhangai ................ 852
13.  ulaanbaatar .............. 767
14.  the ...................... 752
15.  mercado .................. 733
16.  camping .................. 709
17.  достопримечательность .... 695
18.  продукты ................. 670
19.  market ................... 668
20.  petrol ................... 647
21.  piazza ................... 644
22.  ulaan .................... 631
23.  saint .................... 627
24.  airp ..................... 619
25.  гостиница ................ 616
26.  где поесть ............... 615
27.  ugiinnuur ................ 579
28.  zara .....................